In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [6]:
from neural_language_model import TextDataset, dataloader, ft_embedding, NLM  # some issue with this

from neural_language_model.preprocess_data import data_filepath

### Hide

In [ ]:
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.is_available()

In [ ]:
# import zipfile
# path_to_zip_file = '/scratch/hmnshpl/a
# anlp_data/glove.6B.zip'
# directory_to_extract_to = '/scratch/hmnshpl/anlp_data'
# zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
# zip_ref.extractall(directory_to_extract_to)
# zip_ref.close()

In [ ]:
print(f'loading embedding', end='\r')
embedding_model = ft_embedding.load_glove_embeddings()
print(f'loaded embedding     ')

file_path = '/scratch/hmnshpl/anlp_data/Auguste_Maquet.txt'  # Example path to dataset file
embedding_dim = 300  # Make sure it matches the dimensionality of your pre-trained embeddings

# Create the dataset instance
dataset = TextDataset(file_path, embedding_model, embedding_dim=embedding_dim)

# Create DataLoader
batch_size = 32  # Example batch size
train_loader = create_dataloaders(dataset, batch_size=batch_size)

# Example of iterating through the DataLoader
for batch in train_loader:
    contexts, targets = batch
    print("Contexts Shape:", contexts.shape)  # Should be (batch_size, context_size, embedding_dim)
    print("Targets Shape:", targets.shape)    # Should be (batch_size,)
    break  # Remove this line to go through the whole dataset


### Building a neural model

In [ ]:
import os
from torch.utils.data import random_split
from collections import Counter
from sklearn.model_selection import train_test_split

scratch_location = '/scratch/hmnshpl/anlp_data'
filename = 'Auguste_Maquet.txt'
filepath = os.path.join(scratch_location, filename)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
glove_file_path = '/scratch/hmnshpl/anlp_data/glove.6B.300d.txt' 
embedding_dim = 300

embedding_model = ft_embedding.load_glove_embeddings(glove_file_path, embedding_dim)

In [ ]:
# Create dataset
full_dataset = TextDataset(filepath, embedding_model)

# Split dataset
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])


In [ ]:
# Model parameters
vocab_size = len(full_dataset.vocab)
embedding_dim = full_dataset.embedding_dim
hidden_dims = [300, 200]

print(vocab_size, embedding_dim)

In [ ]:
# Initialize model
print('Initializing model', end='\r')
model = NLM.NNLM(vocab_size, embedding_dim, hidden_dims[-1])
print('Model Initialized    ')


# Train model
print('Training model', end='\r')
model.train_model(train_dataset, val_dataset, num_epochs=10, learning_rate=0.001)
print('model training done')


# Evaluate
print('Evaluating model', end=' ')
test_perplexity = model.perplexity(test_dataset)
print(f"Test Perplexity: {test_perplexity}")

# Make a prediction
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
sample_context, _ = next(iter(test_loader))
prediction = model.predict(sample_context)
predicted_word_idx = prediction.argmax().item()
predicted_word = full_dataset.idx_to_word[predicted_word_idx]
print(f"Predicted word: {predicted_word}")

### Building Transformer